## HW2

> using efficientNet

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
# see more data augmentation https://pytorch.org/vision/stable/transforms.html
mean = (0.5071, 0.4867, 0.4408)
std = (0.2675, 0.2565, 0.2761)
train_transform = transforms.Compose(
    [transforms.RandomHorizontalFlip(p=0.5),
     transforms.ToTensor(),
     transforms.Normalize(mean, std)]) # calculte yourself

test_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(mean, std)]) # calculte yourself

batch_size = 64
num_classes = 100    # check

trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                        download=True, transform=train_transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=test_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)


  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


In [5]:
# pick one

# 1. model defined by yourself
# model = my_CNN()        
   
# 2. off-the-shelf model
# see https://pytorch.org/vision/stable/models.html
# nn.Linear https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.Linear
model = models.efficientnet_b5(pretrained=True) 
# model.classifier = torch.nn.Linear(1408, num_classes) #b2
# model.classifier = torch.nn.Linear(1536, num_classes) #b3
# model.classifier = torch.nn.Linear(1792, num_classes) #b4
model.features[0][0] = nn.Conv2d(3,48,kernel_size=(3,3), stride=(1,1), padding=(1,1), bias=False)
model.classifier[1] = torch.nn.Linear(2048, num_classes)
# model.features.pool0 = nn.Identity()

model.to(device)

EfficientNet(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(48, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): ConvNormActivation(
            (0): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
            (1): BatchNorm2d(48, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): ConvNormActivatio

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9, weight_decay=1e-4)

In [7]:
total_epoch = 200
print_per_iteration = 100
acc_list = []
max_acc = (0,0)

for epoch in range(total_epoch):  # loop over the dataset multiple times
    model.train()
    save_path = '/content/drive/MyDrive/Colab Notebooks/ML_HW2/myCNN_test/efficientnet_b5.pth'
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        # print(inputs)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        # if (i+1) % print_per_iteration == 0:    # print every 2000 mini-batches
        #     print(f'[ep {epoch + 1}][{i + 1:5d}/{len(trainloader):5d}] loss: {loss.item():.3f}')
    # torch.save(model, save_path)

    correct = 0
    total = 0
    model.eval()
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            # calculate outputs by running images through the network
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            n_acc = correct/total
    print(f'Accuracy of epoch {epoch+1} : {100 * correct / total:.2f} %')
    acc_list.append(n_acc)
      
    if n_acc > max_acc[0] :
      max_acc = (n_acc,epoch)
      torch.save(model, save_path)

Accuracy of epoch 1 : 1.67 %
Accuracy of epoch 2 : 1.64 %
Accuracy of epoch 3 : 2.80 %
Accuracy of epoch 4 : 4.08 %
Accuracy of epoch 5 : 4.73 %
Accuracy of epoch 6 : 5.78 %
Accuracy of epoch 7 : 6.63 %
Accuracy of epoch 8 : 7.87 %
Accuracy of epoch 9 : 8.44 %
Accuracy of epoch 10 : 9.37 %
Accuracy of epoch 11 : 10.25 %
Accuracy of epoch 12 : 11.34 %
Accuracy of epoch 13 : 12.56 %
Accuracy of epoch 14 : 13.57 %
Accuracy of epoch 15 : 14.22 %
Accuracy of epoch 16 : 15.35 %
Accuracy of epoch 17 : 16.80 %
Accuracy of epoch 18 : 17.93 %
Accuracy of epoch 19 : 18.56 %
Accuracy of epoch 20 : 19.79 %
Accuracy of epoch 21 : 21.01 %
Accuracy of epoch 22 : 21.90 %
Accuracy of epoch 23 : 22.66 %
Accuracy of epoch 24 : 23.83 %
Accuracy of epoch 25 : 24.66 %
Accuracy of epoch 26 : 25.68 %
Accuracy of epoch 27 : 26.69 %
Accuracy of epoch 28 : 27.64 %
Accuracy of epoch 29 : 28.62 %
Accuracy of epoch 30 : 29.75 %
Accuracy of epoch 31 : 30.65 %
Accuracy of epoch 32 : 31.71 %
Accuracy of epoch 33 : 32.5

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x = range(20,200,1)
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['figure.dpi'] = 300
plt.plot(x,acc_list)
plt.xticks(np.arange(0,220,step=20))
plt.title('EfficientNet_b3')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.grid(color='black', linestyle='-', linewidth='1')
plt.show()

print(max_acc)

NameError: ignored